# Workshop 2 Exercise 2 

# Handwritten Digit Recognition

## Neural Networks Basics

<img src=http://www1bpt.bridgeport.edu/~psuksing/nnbg3.gif width="400">

1. Input neurons represent an input based on the information we are trying to classify
2. Each number in the input neurons is given a weight at each synapse
3. At each neuron in the next layer, we add the outputs of all synapses coming to that neuron along with a bias and apply an **activation function** to the weighted sum
4. The output of that function will be treated as an input for the next layer
5. Continue until you reach the output

<img src=https://www.kdnuggets.com/wp-content/uploads/simple-neural-network.png width="500">

An activation function is a function that is used to get the output of a node. It is also known as a transfer function. It maps the resulting values between 0 to 1 or -1 to 1 (depending upon the function)

The sigmoid activaiton function is often used when we are predicting the probability of something. The sigmoid function, just like the probability of something, exists between 0 and 1. 

The Rectified Linear Unit (ReLU) is the most used activation function right now. It is used in almost all the convolutional neural networks

There are many activation functions. Some popular activation functions are

<img src=https://miro.medium.com/max/1838/1*RD0lIYqB5L2LrI2VTIZqGw.png width="500">


You can read more about activation functions and their uses here
https://en.wikipedia.org/wiki/Activation_function




# MNIST Dataset description

- The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples. 
- It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. 
- Each digit has dimension 28x28 (i.e. 768) pixels, and is represented in grayscale.
- There are only 10 digits (0 to 9) and classes to predict.
- Results are reported using prediction error (i.e. inverted classification accuracy).
- Excellent results have a prediction error of less than 1%, and superb results with errors of 0.2% can be achieved with large Convolutional Neural Networks

It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

In [0]:
# We can use Keras to download the MNIST dataset for us! Let's see what
# it looks like:
%matplotlib inline

# Plot ad hoc mnist instances
from keras.datasets import mnist
import matplotlib.pyplot as plt


# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# plot 4 images as gray scale
plt.subplot(221)
plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(X_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(X_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(X_train[3], cmap=plt.get_cmap('gray'))
plt.show()

# Applying MNIST on a simple Neural Network (1 hidden layer)

In [0]:
# setup
import numpy
import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

# seed our random number for reproducibility
seed = 314159
random.seed(seed)

# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Since we're training a NN with pixels as input, we have to linearize / flatten our images. Hence, we'll be creating an input layer of 28x28 or 768 input neurons.

In [0]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

Since these images are greyscale, **the pixel values are from 0 to 255**. As-is, we can apply this data on the NN, but it's a good idea to standardize the data by normalizing it. In this case, since it is well-known, we normalize by the maximimum possible pixel value, 255.

*In theory, it's not necessary to normalize numeric x-data (also called independent data). However, practice has shown that when numeric x-data values are normalized, neural network training is often more efficient, which leads to a better predictor. Basically, if numeric data is not normalized, and the magnitudes of two predictors are far apart, then a change in the value of a neural network weight has far more relative influence on the x-value with larger magnitudes.*

See: https://visualstudiomagazine.com/articles/2014/01/01/how-to-standardize-data-for-neural-networks.aspx

In [0]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# we can verify that scaling has occurred:
print(X_train.max())  # returns 1
print(X_test.max())  # returns 1

Since our output variable will contain the numbers 0 - 9, this is a multi-class classification problem. So, it's a good idea to transform the class integers into a binary matrix (i.e. "hot encoding").

In [0]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

Now we define the model of the Neural Network (which can be adapted to improve the score):

In [0]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, 
                    input_dim=num_pixels, 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(num_classes, 
                    kernel_initializer='normal', 
                    activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

The model is a simple neural network with one hidden layer with the same number of neurons as there are inputs (784). A rectifier activation function is used for the neurons in the hidden layer.

A softmax activation function is used on the output layer to turn the outputs into probability-like values and allow one class of the 10 to be selected as the model’s output prediction. Logarithmic loss is used as the loss function (called categorical_crossentropy in Keras) and the efficient ADAM gradient descent algorithm is used to learn the weights.

We can now fit and evaluate the model. The model is fit over 10 epochs with updates every 200 images. The test data is used as the validation dataset, allowing you to see the skill of the model as it trains. A verbose value of 2 is used to reduce the output to one line for each training epoch.

Now, we can use the test dataset and evaluate the model and print the classification error rate.

In [0]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

# Simple Convolutional Neural Network for MNIST

Now that we have seen how to load the MNIST dataset and train a simple multi-layer perceptron model on it, it is time to develop a more sophisticated convolutional neural network or CNN model.

Keras does provide a lot of capability for creating convolutional neural networks.

In this section we will create a simple CNN for MNIST that demonstrates how to use all of the aspects of a modern CNN implementation, including Convolutional layers, Pooling layers and Dropout layers.

The first step is to import the classes and functions needed.

In [0]:
# setup imports and random number
import numpy
import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

# set a seed for reproducibility:
seed = 314159
random.seed(seed)

Next we need to load the MNIST dataset and reshape it so that it is suitable for use training a CNN. In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions `[pixels][width][height]`.

In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In the case of MNIST where the pixel values are gray scale, the pixel dimension is set to 1.

In [0]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

As before, it is a good idea to normalize the pixel values to the range 0 and 1 and one hot encode the output variables.

In [0]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

### What is a Convolutional Neural Network (CNN)?
Images are represented as matrices of numbers. For greyscale images these numbers range from 0 to 255. Matrices are convolved to created a feature map. For example a 7x7 matrix convolved with a 3x3 matrix may look like

![alt text](https://anhvnn.files.wordpress.com/2018/02/convolve.png?w=1000)

The type of convolution applied depends on the desired outcome.

![alt text](http://i1.wp.com/ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-05-at-11-03-00-pm.png?resize=342%2C562&ssl=1)

1D convolutions are sometimes used before Recurrent Neural Networks (RNNs) because they decrease the amount of data visible by the RNN. This is a preprocessing technique that makes sequences smaller by downsampling. After the 1D convolution, only higher level features are visible by the RNN.

Convolutional neural networks are more complex than standard multi-layer perceptrons, so we will start by using a simple structure to begin with that uses all of the elements for state of the art results. Below summarizes the network architecture.

1. The first hidden layer is a **convolutional layer** called a Convolution2D. The layer has 32 feature maps (5x5 filter matrices) and a rectifier activation function. This is the input layer, expecting images with the structure outline above `[pixels][width][height]`.
2. Next we define a **pooling layer** that takes the max called MaxPooling2D. It is configured with a pool size of 2×2.
<img src=https://qph.fs.quoracdn.net/main-qimg-95aaf9084b7f1fe5eec3f37b1d8b89d3 width="900">
https://qph.fs.quoracdn.net/main-qimg-95aaf9084b7f1fe5eec3f37b1d8b89d3
3. The next layer is a regularization layer using **dropout** called Dropout. It is configured to randomly exclude 20% of neurons in the layer in order to reduce overfitting.
4. Next is a layer that converts the 2D matrix data to a vector called Flatten. It allows the output to be processed by standard fully connected layers.
5. Next a **fully connected layer** with 128 neurons and rectifier activation function. A fully connected layer connects every neuron in one layer to every neuron in another layer
6. Finally, the output layer has 10 neurons for the 10 classes and a softmax activation function to output probability-like predictions for each class.

example of a convolutional neural network format

<img src=https://www.easy-tensorflow.com/images/NN/img1.png width="900">


As before, the model is trained using logarithmic loss and the ADAM gradient descent algorithm.

In [0]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, 
                     (5, 5), 
                     input_shape=(1, 28, 28), 
                     activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

Next, we evaluate the model the same way as before with the multi-layer perceptron. The CNN is fit over 10 epochs with a batch size of 200.

The accuracy on the training and validation test is printed each epoch and at the end of the classification error rate is printed.

In [0]:
# go to Runtime and change runtime type and select GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
model = baseline_model()

# Fit the model using the GPU
with tf.device('/gpu:0'):
  model.fit(X_train, 
          y_train, 
          validation_data=(X_test, y_test), 
          epochs=10, 
          batch_size=200, 
          verbose=2)

  # Final evaluation of the model
  scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

# Larger Convolutional Neural Network for MNIST

Now that we have seen how to create a simple CNN, let’s take a look at a model capable of close to state of the art results.

We import classes and function then load and prepare the data the same as in the previous CNN example.

In [0]:
# Larger CNN for the MNIST Dataset
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

# fix random seed for reproducibility
seed = 314159
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

This time we define a large CNN architecture with additional convolutional, max pooling layers and fully connected layers. The network topology can be summarized as follows.

1. Convolutional layer with 30 feature maps of size 5×5.
2. Pooling layer taking the max over 2*2 patches.
3. Convolutional layer with 15 feature maps of size 3×3.
4. Pooling layer taking the max over 2*2 patches.
5. Dropout layer with a probability of 20%.
6. Flatten layer.
7. Fully connected layer with 128 neurons and rectifier activation.
8. Fully connected layer with 50 neurons and rectifier activation.
9. Output layer.

In [0]:
# define the larger model
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, 
                     (5, 5), 
                     input_shape=(1, 28, 28), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

Like the previous two experiments, the model is fit over 10 epochs with a batch size of 200.

Running the example prints accuracy on the training and validation datasets each epoch and a final classification error rate.

In [0]:
# build the model
model = larger_model()

# Fit the model using the available GPU
with tf.device('/gpu:0'):
  model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

  # Final evaluation of the model
  scores = model.evaluate(X_test, y_test, verbose=0)

print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

This slightly larger model takes a bit longer per epoch than the previous, and achieves the respectable classification error rate of 0.76%.

# Activity 1: How does the classification error change if you train the NN using the unnormalized data?

# Activity 2: How does the choice of activation function affect the NN and its classification error rate?

Hint: Keras supports the following activation functions: `softplus`, `softsign`, `softsign`, `relu`, `tanh`, `sigmoid`, `hard_sigmoid`, and `linear`. See: https://keras.io/activations/

# Activity 3: Randomly choose some test data, evaluate it using the NN, and visualize their classifications

# Reference: 
1. https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/